In [1]:
# building a LSTM VAE model fro scratch
import numpy as np
import tensorflow as tf
from tensorflow.nn.rnn_cell import MultiRNNCell, LSTMCell
from utils import Data_Hanlder


def lrelu(x, leak=0.2, name='lrelu'):
	return tf.maximum(x, leak*x)
def _LSTMCells(unit_list,act_fn_list):
    return MultiRNNCell([LSTMCell(unit,                         
                         activation=act_fn) 
                         for unit,act_fn in zip(unit_list,act_fn_list )])


class LSTM_VAE(object):
    def __init__(self,dataset_name,columns,z_dim,time_steps,outlier_fraction):
        self.outlier_fraction = outlier_fraction # the outlier fraction to be used later on
        self.data_source = Data_Hanlder(dataset_name,columns,time_steps)
        self.n_hidden = 16 # number of hidden units in decoder LSTM
        self.batch_size = 128
        self.learning_rate = 0.0005
        self.train_iters = 4000
        
        self.input_dim = len(columns) # input dimension of x,denoted as m
        self.z_dim = z_dim # dimension of z 
        self.time_steps = time_steps # number of the time steps to consider
    
        self.pointer = 0 
        self.anomaly_score = 0
        self.sess = tf.Session()
        self._build_network()
        self.sess.run(tf.global_variables_initializer())
        
    def _build_network(self):
        with tf.variable_scope('ph'):
            self.X = tf.placeholder(tf.float32, shape = [None, self.time_steps,self.input_dim], name = 'input_X')
            # self.X will have name ph.input_X, the input shape is batch size, time steps, input dimension
        with tf.variable_scope('encoder'):
            with tf.variable_scope('lat_mu'):
                mu_fw_cells = _LSTMCells([self.z_dim],[lrelu]) 
                #z_dim: is the number of units (this is the output size), mu_fw_cells has dimension z_dim
                mu_bw_cells = _LSTMCells([self.z_dim],[lrelu])
                (mu_fw_outputs,mu_fw_outputs),_ = tf.nn.bidirectional_dynamic_rnn(mu_fw_cells,
                                                                                  mu_bw_cells,
                                                                                  self.X,
                                                                                  dtype=tf.float32)
                    # the output size is z_dim, input is self.X which x has shape batch_size*time_steps*#of features
                    # the _ in the left side is the for hidden state
                mu_outputs = tf.add(mu_fw_outputs+mu_fw_outputs)
                
            with tf.variable_scope('lat_sigma'):
                sigma_fw_lstm_cells = _LSTMCells([self.z_dim],[tf.nn.softplus]) # why here is softplus
                sigma_bw_lstm_cells = _LSTMCells([self.z_dim],[tf.nn.softplus])
                (sigma_fw_outputs,sigma_bw_outputs),_ = tf.nn.bidirectional_dynamic_rnn(
                                                            sigma_fw_lstm_cells,
                                                            sigma_bw_lstm_cells, 
                                                            self.X, dtype=tf.float32)
                sigma_outputs = tf.add(sigma_fw_outputs,sigma_bw_outputs) 
                sample_Z = mu_outputs + mu_outputs* tf.random_normal(
                                                        tf.shape(mu_outputs),
                                                        0,1,dtype=tf.float32)  # construnct z with random variable
            with tf.variable_scope('decoder'):
                # below adpot a stacked LSTM structure (think about one LSTM cell stacked on top of the other LSTM cell), 
                # the input is sample_z, the hidden state of the first layer is self.n_hidden
                # the hidden state of the 2nd layer is self.input_dim, the output shape is Batch size*input_dim
                recons_lstm_cells = _LSTMCells([self.n_hidden,self.input_dim],[lrelu,lrelu])
                self.reconst_X, _= tf.nn.dynamic_rnn(recons_lstm_cells,sample_Z, dtype=tf.float32)

            # construct the loss part 
            with tf.variable_scope('loss'):
                reduce_dims = np.arange(1,tf.keras.backend.ndim(self.X)) # if x has 3 dim, reduce_dims would be [1,2,3]
                recons_loss = tf.losses.mean_squared_error(self.X, self.recons_X) # reconstruction loss
                kl_loss = - 0.5 * tf.reduce_mean(1 + sigma_outputs - tf.square(mu_outputs) - tf.exp(sigma_outputs))# KL divergence
                self.opt_loss = recons_loss + kl_loss
                # all lossed queeze the loss in 3 dimensions to 1 dimension?
                self.all_losses = tf.reduce_sum(tf.square(self.X - self.recons_X),reduction_indices=reduce_dims)

            with tf.variable_scope('train'):
                # this returns a operation that update the variable
                self.uion_train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.opt_loss)
                
                        
    def train(self):
        for i in range(self.train_iters):            
            this_X = self.data_source.fetch_data(self.batch_size) # get data in batches, one after the other
            self.sess.run([self.uion_train_op],feed_dict={
                    self.X: this_X
                    }) # execute the opertion (doing gradiednt descent) with the placeholder filled with input data
            if i % 200 ==0:
                mse_loss = self.sess.run([self.opt_loss],feed_dict={
                    self.X: self.data_source.train
                    }) # for every 200 iterations, we want to evaluate the opt_loss
                print('round {}: with loss: {}'.format(i,mse_loss))
        self._arange_score(self.data_source.train) #  

    def _arange_score(self,input_data):
        # this function is used to generate anomaly score
        input_all_losses = self.sess.run(self.all_losses,feed_dict={
                self.X: input_data                
                }) #evaluate all losse in this part 
        # now there is a designate fraction of the data treated to be anomaly, 
        # np.percetile(x,a%) provide value y, s.t. a% of x is greater than y 
        # in this case, any value that is bigger than self.anomaly_score is considered as anoamly
        self.anomaly_score = np.percentile(input_all_losses,(1-self.outlier_fraction)*100)
        
    def judge(self,test):
        all_test_loss = self.sess.run(self.all_losses,feed_dict={
                                    self.X: test                
                                    })
        result = map(lambda x: 1 if x< self.anomaly_score else -1,all_test_loss)

        return list(result)

    def plot_confusion_matrix(self):
        predict_label = self.judge(self.data_source.test)
        self.data_source.plot_confusion_matrix(self.data_source.test_label,predict_label,['Abnormal','Normal'],'LSTM_VAE Confusion-Matrix')

    
def main():
    lstm_vae = LSTM_VAE('dataset/data0.csv',['v0','v1'],z_dim=8,time_steps=16,outlier_fraction=0.01)    
    lstm_vae.train()  
    lstm_vae.plot_confusion_matrix() 

if __name__ == '__main__':
    main()
        
        
        
        
        
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-1-c6bd4c46ac30>, line 55)

In [9]:

import tensorflow as tf;  
import numpy as np;  
import matplotlib.pyplot as plt;  
 
v = tf.Variable(tf.constant(0.0, shape=[1], dtype=tf.float32), name='v')
v1 = tf.Variable(tf.constant(5, shape=[1], dtype=tf.float32), name='v1')
 
global_step = tf.Variable(tf.constant(5, shape=[1], dtype=tf.float32), name='global_step', trainable=False)
ema = tf.train.ExponentialMovingAverage(0.99, global_step)
 
for ele1 in tf.trainable_variables():
	print(ele1.name)


v:0
v1:0
v_1:0
v1_1:0
v_2:0
v1_2:0
